In [138]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
import seaborn as sns
#Baseline
from sklearn.linear_model import LogisticRegression

#Metrics
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, roc_curve, roc_auc_score, precision_recall_curve, auc
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2, f_classif

# For matrics
from scipy.sparse import csr_matrix

# Factorization
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

2
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from ast import literal_eval

In [2]:
train = pd.read_csv('data_train.csv').drop('Unnamed: 0', axis=1)
test = pd.read_csv('data_test.csv').drop('Unnamed: 0', axis=1)

In [3]:
feat = pd.read_csv('features.csv', sep='\t', header=0).drop('Unnamed: 0', axis=1)

In [4]:
USER_COL = 'id'
SERVICE_COL = 'vas_id'
ACTUAL_COL = 'bought_service'

In [5]:
common_users = train[USER_COL].unique()
RANDOM_STATE = 21
train_users = train.id.tolist()
test_users = test.id.tolist()

In [7]:
X_1 = train.merge(feat, on='id', how='left')
# X_2 = test.merge(feat, on='id', how='left')

In [8]:
X_1 = X_1.drop(['buy_time_x', 'buy_time_y'], axis=1)
# X_2 = X_2.drop(['buy_time_x', 'buy_time_y'], axis=1)

In [12]:
n_unique_v_of_feature = X_1.nunique()

In [13]:
f_all = set(n_unique_v_of_feature.index.tolist())
len(f_all)

256

In [14]:
f_const = set(n_unique_v_of_feature[n_unique_v_of_feature == 1].index.tolist())
len(f_const)

5

In [15]:
f_numeric = set(n_unique_v_of_feature[n_unique_v_of_feature >= 10].index.tolist())
len(f_numeric)

221

In [63]:
f_numeric.remove('id')

In [39]:
f_cat = set(n_unique_v_of_feature[n_unique_v_of_feature < 10].index.tolist())
len(f_cat)

35

In [59]:
f_cat.remove('target')

In [60]:
f_cat

{'131',
 '132',
 '139',
 '154',
 '192',
 '194',
 '195',
 '196',
 '197',
 '198',
 '199',
 '200',
 '201',
 '202',
 '203',
 '204',
 '205',
 '206',
 '216',
 '218',
 '23',
 '26',
 '27',
 '29',
 '30',
 '31',
 '32',
 '33',
 '57',
 '75',
 '81',
 '85',
 '95',
 'vas_id'}

In [40]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.columns]


In [1]:
cat_pipe = Pipeline([
    ('selector', ColumnSelector(f_cat)),
    ('imputer', SimpleImputer(strategy="most_frequent")), 
])

num_pipe = Pipeline([
    ('selector', ColumnSelector(f_numeric)),
    ('imputer', SimpleImputer(strategy="mean")),
    ('scaler', StandardScaler())
])

preprocessor = FeatureUnion([
    ('cat', cat_pipe),
    ('num', num_pipe)
])

NameError: name 'Pipeline' is not defined

In [45]:
y = X_1['target']
X = X_1.drop(['id', 'target'], axis=1)

In [46]:
train_idx, test_idx = train_test_split(X.index, test_size=0.25, random_state=RANDOM_STATE)

y_train, y_test = y.loc[train_idx], y.loc[test_idx]
X_train, X_test = X.loc[train_idx], X.loc[test_idx]

print("Train")
print(y.loc[train_idx].value_counts()/train_idx.shape[0])
print()
print("Test")
print(y.loc[test_idx].value_counts()/test_idx.shape[0])

Train
0.0    0.92757
1.0    0.07243
Name: target, dtype: float64

Test
0.0    0.927972
1.0    0.072028
Name: target, dtype: float64


In [115]:
model_params = []
model_predict = []
model_result = []
for i in [1000, 1500]:
    for d in [10, 15]:
        for l in [0.05, 0.005, 0.0005]:
                pipe = Pipeline([('pre', preprocessor),
                                ('model', CatBoostClassifier(iterations=i, depth=d, learning_rate=l, loss_function='Logloss', l2_leaf_reg=1 ))
                                ])
                pipe.fit(X_train, y_train)
                model_params.append([('iterations', i), ('depth', d), ('learning_rate', l), ('l2_leaf_reg', 1)])
                model_predict.append(pipe.predict_proba(X_test)[:,1])
                result = f1_score(y_test, pipe.predict(X_test), average='macro')
                model_result.append(result)
                print()
                print(result)
                print()
                print(50 * '-')
                print()
                


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 0.6170846	total: 632ms	remaining: 10m 31s
1:	learn: 0.5528290	total: 1.23s	remaining: 10m 16s
2:	learn: 0.4983134	total: 1.8s	remaining: 9m 57s
3:	learn: 0.4521355	total: 2.43s	remaining: 10m 4s
4:	learn: 0.4210654	total: 3.02s	remaining: 10m
5:	learn: 0.3867104	total: 3.61s	remaining: 9m 57s
6:	learn: 0.3641166	total: 4.17s	remaining: 9m 50s
7:	learn: 0.3402977	total: 4.7s	remaining: 9m 42s
8:	learn: 0.3184002	total: 5.22s	remaining: 9m 35s
9:	learn: 0.3044384	total: 5.81s	remaining: 9m 35s
10:	learn: 0.2922921	total: 6.4s	remaining: 9m 35s
11:	learn: 0.2794149	total: 6.87s	remaining: 9m 26s
12:	learn: 0.2666795	total: 7.39s	remaining: 9m 21s
13:	learn: 0.2589839	total: 7.98s	remaining: 9m 22s
14:	learn: 0.2522501	total: 8.55s	remaining: 9m 21s
15:	learn: 0.2432638	total: 9.09s	remaining: 9m 18s
16:	learn: 0.2357490	total: 9.6s	remaining: 9m 14s
17:	learn: 0.2293039	total: 10.1s	remaining: 9m 13s
18:	learn: 0.2239228	total: 10.7s	remaining: 9m 12s
19:	learn: 0.2192750	total:

158:	learn: 0.1807373	total: 1m 24s	remaining: 7m 29s
159:	learn: 0.1807240	total: 1m 25s	remaining: 7m 29s
160:	learn: 0.1807126	total: 1m 26s	remaining: 7m 28s
161:	learn: 0.1806877	total: 1m 26s	remaining: 7m 28s
162:	learn: 0.1806287	total: 1m 27s	remaining: 7m 27s
163:	learn: 0.1806037	total: 1m 27s	remaining: 7m 27s
164:	learn: 0.1805782	total: 1m 28s	remaining: 7m 26s
165:	learn: 0.1805284	total: 1m 28s	remaining: 7m 25s
166:	learn: 0.1805134	total: 1m 29s	remaining: 7m 25s
167:	learn: 0.1804745	total: 1m 29s	remaining: 7m 24s
168:	learn: 0.1804632	total: 1m 30s	remaining: 7m 24s
169:	learn: 0.1804482	total: 1m 30s	remaining: 7m 23s
170:	learn: 0.1804103	total: 1m 31s	remaining: 7m 23s
171:	learn: 0.1803838	total: 1m 32s	remaining: 7m 23s
172:	learn: 0.1803733	total: 1m 32s	remaining: 7m 22s
173:	learn: 0.1803607	total: 1m 33s	remaining: 7m 22s
174:	learn: 0.1803350	total: 1m 33s	remaining: 7m 21s
175:	learn: 0.1803165	total: 1m 34s	remaining: 7m 21s
176:	learn: 0.1802851	total:

311:	learn: 0.1764500	total: 2m 47s	remaining: 6m 9s
312:	learn: 0.1764349	total: 2m 48s	remaining: 6m 8s
313:	learn: 0.1763824	total: 2m 48s	remaining: 6m 8s
314:	learn: 0.1763615	total: 2m 49s	remaining: 6m 7s
315:	learn: 0.1763155	total: 2m 49s	remaining: 6m 7s
316:	learn: 0.1762987	total: 2m 50s	remaining: 6m 6s
317:	learn: 0.1762747	total: 2m 50s	remaining: 6m 6s
318:	learn: 0.1762556	total: 2m 51s	remaining: 6m 5s
319:	learn: 0.1762324	total: 2m 51s	remaining: 6m 5s
320:	learn: 0.1762148	total: 2m 52s	remaining: 6m 4s
321:	learn: 0.1761819	total: 2m 52s	remaining: 6m 4s
322:	learn: 0.1761603	total: 2m 53s	remaining: 6m 3s
323:	learn: 0.1761166	total: 2m 54s	remaining: 6m 3s
324:	learn: 0.1760764	total: 2m 54s	remaining: 6m 2s
325:	learn: 0.1760437	total: 2m 55s	remaining: 6m 2s
326:	learn: 0.1760045	total: 2m 55s	remaining: 6m 1s
327:	learn: 0.1759791	total: 2m 56s	remaining: 6m 1s
328:	learn: 0.1759240	total: 2m 56s	remaining: 6m
329:	learn: 0.1758966	total: 2m 57s	remaining: 6m

465:	learn: 0.1724376	total: 4m 11s	remaining: 4m 48s
466:	learn: 0.1724062	total: 4m 11s	remaining: 4m 47s
467:	learn: 0.1723796	total: 4m 12s	remaining: 4m 47s
468:	learn: 0.1723517	total: 4m 13s	remaining: 4m 46s
469:	learn: 0.1723140	total: 4m 13s	remaining: 4m 45s
470:	learn: 0.1722956	total: 4m 14s	remaining: 4m 45s
471:	learn: 0.1722762	total: 4m 14s	remaining: 4m 44s
472:	learn: 0.1722621	total: 4m 15s	remaining: 4m 44s
473:	learn: 0.1722437	total: 4m 15s	remaining: 4m 43s
474:	learn: 0.1722188	total: 4m 16s	remaining: 4m 43s
475:	learn: 0.1722067	total: 4m 16s	remaining: 4m 42s
476:	learn: 0.1721910	total: 4m 17s	remaining: 4m 42s
477:	learn: 0.1721653	total: 4m 17s	remaining: 4m 41s
478:	learn: 0.1721478	total: 4m 18s	remaining: 4m 41s
479:	learn: 0.1721332	total: 4m 19s	remaining: 4m 40s
480:	learn: 0.1721082	total: 4m 19s	remaining: 4m 40s
481:	learn: 0.1720970	total: 4m 20s	remaining: 4m 39s
482:	learn: 0.1720411	total: 4m 20s	remaining: 4m 39s
483:	learn: 0.1720033	total:

...

812:	learn: 0.3389102	total: 1h 9m 6s	remaining: 58m 24s
813:	learn: 0.3386752	total: 1h 9m 12s	remaining: 58m 19s
814:	learn: 0.3384452	total: 1h 9m 17s	remaining: 58m 14s
815:	learn: 0.3382721	total: 1h 9m 22s	remaining: 58m 9s
816:	learn: 0.3380996	total: 1h 9m 28s	remaining: 58m 4s
817:	learn: 0.3379272	total: 1h 9m 33s	remaining: 57m 59s
818:	learn: 0.3377508	total: 1h 9m 38s	remaining: 57m 54s
819:	learn: 0.3375230	total: 1h 9m 43s	remaining: 57m 49s
820:	learn: 0.3373509	total: 1h 9m 48s	remaining: 57m 43s
821:	learn: 0.3371796	total: 1h 9m 53s	remaining: 57m 38s
822:	learn: 0.3370076	total: 1h 9m 58s	remaining: 57m 33s
823:	learn: 0.3367954	total: 1h 10m 3s	remaining: 57m 28s
824:	learn: 0.3366244	total: 1h 10m 8s	remaining: 57m 23s
825:	learn: 0.3364542	total: 1h 10m 13s	remaining: 57m 18s
826:	learn: 0.3362831	total: 1h 10m 18s	remaining: 57m 13s
827:	learn: 0.3360572	total: 1h 10m 23s	remaining: 57m 8s
828:	learn: 0.3358878	total: 1h 10m 28s	remaining: 57m 2s
829:	learn: 0.3

953:	learn: 0.3138160	total: 1h 21m 4s	remaining: 46m 24s
954:	learn: 0.3136262	total: 1h 21m 10s	remaining: 46m 19s
955:	learn: 0.3134342	total: 1h 21m 15s	remaining: 46m 14s
956:	learn: 0.3132923	total: 1h 21m 20s	remaining: 46m 9s
957:	learn: 0.3131525	total: 1h 21m 25s	remaining: 46m 4s
958:	learn: 0.3130127	total: 1h 21m 31s	remaining: 45m 59s
959:	learn: 0.3128448	total: 1h 21m 36s	remaining: 45m 54s
960:	learn: 0.3126715	total: 1h 21m 41s	remaining: 45m 49s
961:	learn: 0.3125290	total: 1h 21m 46s	remaining: 45m 43s
962:	learn: 0.3123411	total: 1h 21m 51s	remaining: 45m 38s
963:	learn: 0.3122026	total: 1h 21m 56s	remaining: 45m 33s
964:	learn: 0.3120643	total: 1h 22m 2s	remaining: 45m 28s
965:	learn: 0.3119259	total: 1h 22m 7s	remaining: 45m 23s
966:	learn: 0.3117367	total: 1h 22m 12s	remaining: 45m 18s
967:	learn: 0.3115989	total: 1h 22m 17s	remaining: 45m 13s
968:	learn: 0.3114605	total: 1h 22m 23s	remaining: 45m 8s
969:	learn: 0.3112726	total: 1h 22m 28s	remaining: 45m 3s
970:

1092:	learn: 0.2934400	total: 1h 33m 12s	remaining: 34m 42s
1093:	learn: 0.2933230	total: 1h 33m 18s	remaining: 34m 37s
1094:	learn: 0.2931846	total: 1h 33m 23s	remaining: 34m 32s
1095:	learn: 0.2930268	total: 1h 33m 28s	remaining: 34m 27s
1096:	learn: 0.2928696	total: 1h 33m 33s	remaining: 34m 22s
1097:	learn: 0.2927125	total: 1h 33m 38s	remaining: 34m 16s
1098:	learn: 0.2925992	total: 1h 33m 43s	remaining: 34m 11s
1099:	learn: 0.2924419	total: 1h 33m 48s	remaining: 34m 6s
1100:	learn: 0.2922859	total: 1h 33m 54s	remaining: 34m 1s
1101:	learn: 0.2921729	total: 1h 33m 59s	remaining: 33m 56s
1102:	learn: 0.2920555	total: 1h 34m 4s	remaining: 33m 51s
1103:	learn: 0.2919166	total: 1h 34m 9s	remaining: 33m 46s
1104:	learn: 0.2918042	total: 1h 34m 15s	remaining: 33m 41s
1105:	learn: 0.2916922	total: 1h 34m 20s	remaining: 33m 36s
1106:	learn: 0.2915798	total: 1h 34m 25s	remaining: 33m 31s
1107:	learn: 0.2914439	total: 1h 34m 30s	remaining: 33m 26s
1108:	learn: 0.2913293	total: 1h 34m 35s	rem

1230:	learn: 0.2761700	total: 1h 45m 4s	remaining: 22m 57s
1231:	learn: 0.2760560	total: 1h 45m 8s	remaining: 22m 52s
1232:	learn: 0.2759250	total: 1h 45m 14s	remaining: 22m 47s
1233:	learn: 0.2758311	total: 1h 45m 19s	remaining: 22m 42s
1234:	learn: 0.2757368	total: 1h 45m 24s	remaining: 22m 37s
1235:	learn: 0.2756231	total: 1h 45m 30s	remaining: 22m 32s
1236:	learn: 0.2755080	total: 1h 45m 35s	remaining: 22m 26s
1237:	learn: 0.2754156	total: 1h 45m 40s	remaining: 22m 21s
1238:	learn: 0.2753208	total: 1h 45m 45s	remaining: 22m 16s
1239:	learn: 0.2751886	total: 1h 45m 51s	remaining: 22m 11s
1240:	learn: 0.2750963	total: 1h 45m 56s	remaining: 22m 6s
1241:	learn: 0.2750045	total: 1h 46m 1s	remaining: 22m 1s
1242:	learn: 0.2748754	total: 1h 46m 6s	remaining: 21m 56s
1243:	learn: 0.2747450	total: 1h 46m 12s	remaining: 21m 51s
1244:	learn: 0.2746149	total: 1h 46m 17s	remaining: 21m 46s
1245:	learn: 0.2744980	total: 1h 46m 22s	remaining: 21m 41s
1246:	learn: 0.2743679	total: 1h 46m 27s	remai

1368:	learn: 0.2619842	total: 1h 56m 58s	remaining: 11m 11s
1369:	learn: 0.2618759	total: 1h 57m 3s	remaining: 11m 6s
1370:	learn: 0.2617678	total: 1h 57m 9s	remaining: 11m 1s
1371:	learn: 0.2616560	total: 1h 57m 14s	remaining: 10m 56s
1372:	learn: 0.2615793	total: 1h 57m 19s	remaining: 10m 51s
1373:	learn: 0.2614717	total: 1h 57m 24s	remaining: 10m 45s
1374:	learn: 0.2613771	total: 1h 57m 29s	remaining: 10m 40s
1375:	learn: 0.2612814	total: 1h 57m 34s	remaining: 10m 35s
1376:	learn: 0.2612030	total: 1h 57m 39s	remaining: 10m 30s
1377:	learn: 0.2610934	total: 1h 57m 45s	remaining: 10m 25s
1378:	learn: 0.2610009	total: 1h 57m 50s	remaining: 10m 20s
1379:	learn: 0.2609265	total: 1h 57m 50s	remaining: 10m 14s
1380:	learn: 0.2608193	total: 1h 57m 55s	remaining: 10m 9s
1381:	learn: 0.2607100	total: 1h 58m	remaining: 10m 4s
1382:	learn: 0.2606324	total: 1h 58m 5s	remaining: 9m 59s
1383:	learn: 0.2605578	total: 1h 58m 11s	remaining: 9m 54s
1384:	learn: 0.2604649	total: 1h 58m 16s	remaining: 9

In [116]:
model_result

[0.5075084609774824,
 0.4813202135384126,
 0.4813202135384126,
 0.5264520401216227,
 0.48132442005439807,
 0.4813202135384126,
 0.5202141954890721,
 0.4813152086072346,
 0.4813202135384126,
 0.5260094401111342,
 0.4822778322341276,
 0.4813202135384126]

In [117]:
results = pd.read_csv('results.csv')

In [126]:
for i in range(len(model_params)):
    results = results.append({'model_params': model_params[i], 'predicts': model_predict[i], 'f1_score': model_result[i]}, ignore_index=True)

In [127]:
pd.set_option('display.max_colwidth', 100)

In [128]:
results = results.iloc[:-1]

In [172]:
results = results.sort_index(ascending=True)

In [133]:
results.to_csv('results.csv', index=False)

In [173]:
results

model_params  \
0    [('iterations', 100), ('depth', 5), ('learning_rate', 0.03), ('n_estimators', 1)]   
1    [('iterations', 100), ('depth', 5), ('learning_rate', 0.03), ('n_estimators', 2)]   
2    [('iterations', 100), ('depth', 5), ('learning_rate', 0.03), ('n_estimators', 3)]   
3    [('iterations', 100), ('depth', 5), ('learning_rate', 0.01), ('n_estimators', 1)]   
4    [('iterations', 100), ('depth', 5), ('learning_rate', 0.01), ('n_estimators', 2)]   
5    [('iterations', 100), ('depth', 5), ('learning_rate', 0.01), ('n_estimators', 3)]   
6    [('iterations', 100), ('depth', 5), ('learning_rate', 0.05), ('n_estimators', 1)]   
7    [('iterations', 100), ('depth', 5), ('learning_rate', 0.05), ('n_estimators', 2)]   
8    [('iterations', 100), ('depth', 5), ('learning_rate', 0.05), ('n_estimators', 3)]   
9   [('iterations', 100), ('depth', 10), ('learning_rate', 0.03), ('n_estimators', 1)]   
10  [('iterations', 100), ('depth', 10), ('learning_rate', 0.03), ('n_estimators', 2)]   
11  [('iterations', 100), ('depth', 10), ('learning_rate', 0.03), ('n_estimators', 3)]   
12  [('iterations', 100), ('depth', 10), ('learning_rate', 0.01), ('n_estimators', 1)]   
13  [('iterations', 100), ('depth', 10), ('learning_rate', 0.01), ('n_estimators', 2)]   
14  [('iterations', 100), ('depth', 10), ('learning_rate', 0.01), ('n_estimators', 3)]   
15  [('iterations', 100), ('depth', 10), ('learning_rate', 0.05), ('n_estimators', 1)]   
16  [('iterations', 100), ('depth', 10), ('learning_rate', 0.05), ('n_estimators', 2)]   
17  [('iterations', 100), ('depth', 10), ('learning_rate', 0.05), ('n_estimators', 3)]   
18  [('iterations', 100), ('depth', 15), ('learning_rate', 0.03), ('n_estimators', 1)]   
19  [('iterations', 100), ('depth', 15), ('learning_rate', 0.03), ('n_estimators', 2)]   
20  [('iterations', 100), ('depth', 15), ('learning_rate', 0.03), ('n_estimators', 3)]   
21  [('iterations', 100), ('depth', 15), ('learning_rate', 0.01), ('n_estimators', 1)]   
22  [('iterations', 100), ('depth', 15), ('learning_rate', 0.01), ('n_estimators', 2)]   
23  [('iterations', 100), ('depth', 15), ('learning_rate', 0.01), ('n_estimators', 3)]   
24  [('iterations', 100), ('depth', 15), ('learning_rate', 0.05), ('n_estimators', 1)]   
25  [('iterations', 100), ('depth', 15), ('learning_rate', 0.05), ('n_estimators', 2)]   
26  [('iterations', 100), ('depth', 15), ('learning_rate', 0.05), ('n_estimators', 3)]   
27   [('iterations', 500), ('depth', 5), ('learning_rate', 0.03), ('n_estimators', 1)]   
28   [('iterations', 500), ('depth', 5), ('learning_rate', 0.03), ('n_estimators', 2)]   
29   [('iterations', 500), ('depth', 5), ('learning_rate', 0.03), ('n_estimators', 3)]   
30   [('iterations', 500), ('depth', 5), ('learning_rate', 0.01), ('n_estimators', 1)]   
31   [('iterations', 500), ('depth', 5), ('learning_rate', 0.01), ('n_estimators', 2)]   
32   [('iterations', 500), ('depth', 5), ('learning_rate', 0.01), ('n_estimators', 3)]   
33   [('iterations', 500), ('depth', 5), ('learning_rate', 0.05), ('n_estimators', 1)]   
34   [('iterations', 500), ('depth', 5), ('learning_rate', 0.05), ('n_estimators', 2)]   
35   [('iterations', 500), ('depth', 5), ('learning_rate', 0.05), ('n_estimators', 3)]   
36  [('iterations', 500), ('depth', 10), ('learning_rate', 0.03), ('n_estimators', 1)]   
37  [('iterations', 500), ('depth', 10), ('learning_rate', 0.03), ('n_estimators', 2)]   
38  [('iterations', 500), ('depth', 10), ('learning_rate', 0.03), ('n_estimators', 3)]   
39  [('iterations', 500), ('depth', 10), ('learning_rate', 0.01), ('n_estimators', 1)]   
40  [('iterations', 500), ('depth', 10), ('learning_rate', 0.01), ('n_estimators', 2)]   
41  [('iterations', 500), ('depth', 10), ('learning_rate', 0.01), ('n_estimators', 3)]   
42  [('iterations', 500), ('depth', 10), ('learning_rate', 0.05), ('n_estimators', 1)]   
43  [('iterations', 500), ('depth', 10), ('learning_rate', 0.05), ('n_estimators', 2)]   
44  [('iterations', 500)

In [178]:
results.predicts[]

array([0.01859622, 0.02375912, 0.01390123, ..., 0.02057681, 0.02115945,
       0.01577714])

In [174]:
model_predict

12

In [168]:
len(y_test)

215013

In [179]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)

f.set_figheight(24)
f.set_figwidth(12)

ax1 = plt.subplot(211)
ax1.set_title('ROC')
ax1.set_xlabel('TPR')
ax1.set_ylabel('FPR')


ax2 = plt.subplot(212)
ax2.set_title('PR')
ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')

ax1.plot([0, 1], [0, 1], 'k--')

fpr, tpr, roc_thresholds = roc_curve(y_test, results.predicts[56])
precision, recall, pr_thresholds = precision_recall_curve(y_test, results.predicts[56])
ax1.plot(fpr, tpr, label='Top_1', color='b')
ax2.plot(recall, precision, label='Top_1', color='b')
print("Top_1: AUC_PR = %.3f" % auc(recall, precision))
print("Top_1: AUC_ROC = %.3f" % auc(fpr, tpr))


fpr, tpr, roc_thresholds = roc_curve(y_test, results.predicts[52])
precision, recall, pr_thresholds = precision_recall_curve(y_test, results.predicts[52])
ax1.plot(fpr, tpr, label="Top_2", color='g')
ax2.plot(recall, precision, label="Top_2", color='g')
print("Top_2: AUC_PR = %.3f" % auc(recall, precision))
print("Top_2: AUC_ROC = %.3f" % auc(fpr, tpr))


fpr, tpr, roc_thresholds = roc_curve(y_test, results.predicts[49])
precision, recall, pr_thresholds = precision_recall_curve(y_test, results.predicts[49])
ax1.plot(fpr, tpr, label="Top_3", color='purple')
ax2.plot(recall, precision, label="Top_3", color='purple')
print("Top_3: AUC_PR = %.3f" % auc(recall, precision))
print("Top_3: AUC_ROC = %.3f" % auc(fpr, tpr))
ax1.legend(loc="lower right")
ax2.legend(loc="lower right");


Top_1: AUC_PR = 0.329
Top_1: AUC_ROC = 0.835
Top_2: AUC_PR = 0.325
Top_2: AUC_ROC = 0.835
Top_3: AUC_PR = 0.276
Top_3: AUC_ROC = 0.822
